In [3]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import time

In [7]:
# 文件名
# file_name = 'static_0k_out.xyz'
# 文件位置
file_path = r'D:\MD_file\Ge-Si\move_f\glass_300k_out.xyz'
#需要计算配位的元素名称 字典是用来计数这个元素在A附近出现的次数 
all_element_dic = {'O':0,'F':0,'Si':0}

#需要分析的元素以及第一近邻的距离
A_dis = 2.5
name_A = 'Si'

# 分析的起始帧、终止帧、以及步长
start_frame = 0
steps_frame = 1
final_frame = 1

# 是否将分析结果存入文件
save_result = 0
save_path = r'C:\Users\zhang\Desktop\50sap_out\\'
#save_name = file_name[:5] + '_ana'
#save_name+='.txt'
# 是否创建新帧  转换格式
creat_new_frame = 0
out_type = 'xyz'

In [9]:
# 打开文件并读取所有行数据
with open(file_path, 'r') as file:
    lines = file.readlines()
int(lines[0])
# 每一页的原子数
num_atom =int(lines[0])
# 定义一页的行数
lines_per_page = num_atom + 2
# 计算总页数
total_pages = (len(lines) + lines_per_page - 1) // lines_per_page

if final_frame>total_pages:
    print('Input Error!')

In [11]:
def cal_ppp_distance(pos1, pos2, box_size):
    # 计算两个位置之间的最小距离，考虑周期性边界条件
    pos1,pos2,box_size = np.array(pos1),np.array(pos2),np.array(box_size)
    # 计算位置差矢量
    delta = pos2 - pos1
    
    # 应用最小图像约束，确保距离在周期内最短
    delta = delta - np.round(delta / box_size) * box_size
    # 计算距离
    distance = np.linalg.norm(delta)
    # 如果是原子本身
    if distance<1e-5:
        distance=float('inf')
    return distance

In [13]:
def draw_flower():
    pattern = [
        " 🌹🌹🌹  🌹   🌹    🌹 ",
        " 🌹  🌹  🌹  🌹     🌹 ",
        " 🌹  🌹  🌹🌹🌹     🌹 ",
        " 🌹  🌹  🌹   🌹       ",
        " 🌹🌹🌹  🌹    🌹   🌹  "
    ]

    for row in pattern:
        print(row)

In [15]:
# # 记录每一个帧数处理后得到的信息，然后加和
# 得到元素A的数量
num_A_atom = 0
# 记录A总的近邻和Cn数
sum_A_neibor = 0
sum_A_Cn = np.zeros(100)
# 字典是用来计数的
sum_element_dic = copy.deepcopy(all_element_dic)
# 盒子尺寸
box_size = []
# 循环次数
count_cycle = 0 

In [17]:
# 开始计时
start_time = time.time()
# 
max_x,max_y,max_z = 0,0,0
################### 开始循环处理每一帧 #################
for frame in range(start_frame,final_frame,steps_frame):
    
    count_cycle+=1
    
    data = lines[frame*lines_per_page:(frame+1)*lines_per_page]
    ##  去除表头
    data = data[2:]
    ##  先对数据处理一遍
    new_data = []
    ana_element_list = []
#     # 先把数据切片，再分析
#     data = data.split()
#     for i in range(0,len(data),4):
#         temp_x,temp_y,temp_z =data[i+1],data[i+2],data[i+3]
#         max_x,max_y,max_z = max(temp_x,max_x),max(temp_y,max_y),max(temp_z,max_z)
#         new_data.append([atom[0],temp_x,temp_y,temp_z])
    for atom in data:
        # 转换格式
        atom = atom.split()
        temp_x,temp_y,temp_z = float(atom[1]),float(atom[2]),float(atom[3])
        max_x,max_y,max_z = max(temp_x,max_x),max(temp_y,max_y),max(temp_z,max_z)
        new_data.append([atom[0],temp_x,temp_y,temp_z])
        # 如果是要分析的元素则记录下来
        if atom[0] == name_A:
            ana_element_list.append([atom[0],float(atom[1]),float(atom[2]),float(atom[3])])
    ## 全局变量
    box_size = [max_x,250,max_z]
    print(f'此处更改了下boxsize，{box_size}')
    num_A_atom = len(ana_element_list)
    ## 配位数记录的数据表
    # 记录这个A是几配位
    temp_cn_list = np.zeros(100)
    # 每一次要拷贝一次字典 便于重新计数
    frame_element_dic = copy.deepcopy(all_element_dic)
    ## 开始分析
    # 对所有A原子进行分析
    for ana_atom in ana_element_list:
        # 这是分析每一个A原子
        temp_cn = 0
        for neighber_atom in new_data:
            # 只关注于dic中有没有这个元素
            if (neighber_atom[0] not in all_element_dic):
                continue
            atom1 = [ana_atom[1],ana_atom[2],ana_atom[3]]
            atom2 = [neighber_atom[1],neighber_atom[2],neighber_atom[3]]
            if cal_ppp_distance(atom1,atom2,box_size)<A_dis:
                temp_cn+=1
                # 再记录这个近邻的种类 先取出原有值，再加一
                temp = frame_element_dic[neighber_atom[0]]
                frame_element_dic[neighber_atom[0]] = temp+1
        # 本循环结束 记录下这个A元素的Cn值   
        temp_cn_list[temp_cn]+= 1
    #####################已经得到了这一帧的所有数据########################   
    # 第一项 Cn累加
    for i in range(100):
        sum_A_Cn[i] += temp_cn_list[i]
    # 第二项 总的配位数累加
    for i in frame_element_dic:
        # 如果表明不计数 则跳过
        if frame_element_dic[i] == -1:
            continue
        sum_element_dic[i] += frame_element_dic[i]

# 结束计时
end_time = time.time()
# 计算运行时间
run_time = end_time - start_time

# 打印运行时间
print("程序运行时间：", run_time, "秒")

此处更改了下boxsize，[53.8095, 250, 53.8098]
程序运行时间： 103.7117018699646 秒


In [18]:
########  计算完成 输出结果  ##############
print('file_name: ' + file_path.split('\\')[-1])
print('我们关注的是 '+ name_A +' 的配位数，采取的截断距离为 '+str(A_dis))
print(name_A+'有'+ str(num_A_atom) +'个，它的Cn分布如下：')
for i in range(100):
    if sum_A_Cn[i] != 0 :
        temp_x = sum_A_Cn[i]//count_cycle
        pro = temp_x/num_A_atom*100
        print(f'C' +str(i)+': '+str(sum_A_Cn[i]//count_cycle).center(8)+ '    占比： '+str(round(pro,2))+'%')
# 再来分析A的平均配位数
sum_cn = 0
for i in sum_element_dic:
    if sum_element_dic[i] != -1:
        sum_cn += sum_element_dic[i]
print(name_A+'的平均Cn是： '+str(round(sum_cn/(num_A_atom*count_cycle),2)))

cn = round(sum_cn/(num_A_atom*count_cycle),3)

# 分析各个元素的配位占比
print('元素列表中的总体配位情况如下：')
for i in sum_element_dic:
    if sum_element_dic[i]== -1:
        pass
        #print(i + " 元素不计入配位计算！")
    else:
        print(i + ' : '+'配位数量：'+str(sum_element_dic[i]//count_cycle)+' 占比： '+ str(round(sum_element_dic[i]/sum_cn*100,1))+'%',' 均值:',str(round(cn*(sum_element_dic[i]/sum_cn),3)))

file_name: glass_300k_out.xyz
我们关注的是 Si 的配位数，采取的截断距离为 2.5
Si有1876个，它的Cn分布如下：
C0:   2.0       占比： 0.11%
C2:   1.0       占比： 0.05%
C3:   43.0      占比： 2.29%
C4:  1803.0     占比： 96.11%
C5:   26.0      占比： 1.39%
C6:   1.0       占比： 0.05%
Si的平均Cn是： 3.99
元素列表中的总体配位情况如下：
O : 配位数量：6676 占比： 89.3%  均值: 3.559
F : 配位数量：797 占比： 10.7%  均值: 0.425
Si : 配位数量：6 占比： 0.1%  均值: 0.003


In [20]:
# 输出到文件中去
if save_result == 1:
    with open(save_path+save_name,'w') as note:
        note.write('我们关注的是 '+ name_A +' 的配位数，采取的截断距离为 '+str(A_dis)+'\n')
        note.write(name_A+'有'+ str(num_A_atom) +'个，它的Cn分布如下：'+'\n')
        for i in range(50):
            if sum_A_Cn[i] != 0 :
                note.write('C' +str(i)+': '+str(sum_A_Cn[i]//count_cycle)+'\n')
        # 再来分析A的平均配位数
        sum_cn = 0
        for i in sum_element_dic:
            if sum_element_dic[i] != -1:
                sum_cn += sum_element_dic[i]
        note.write(name_A+'的平均Cn是： '+str(round(sum_cn/(num_A_atom*count_cycle),2))+'\n')

        # 分析各个元素的配位占比
        note.write('元素列表中的总体配位情况如下：'+'\n')
        for i in sum_element_dic:
            if sum_element_dic[i]== -1:
                pass
            else:
                note.write(i + ' : '+'配位数量：'+str(sum_element_dic[i]//count_cycle)+' 占比： '+ str(round(sum_element_dic[i]/sum_cn*100,1))+'%'+'\n')
        print('结果已经保存!')
else:
    print('结果没有存储!')

结果没有存储!


In [21]:
# 调用函数绘制小红花
draw_flower()

 🌹🌹🌹  🌹   🌹    🌹 
 🌹  🌹  🌹  🌹     🌹 
 🌹  🌹  🌹🌹🌹     🌹 
 🌹  🌹  🌹   🌹       
 🌹🌹🌹  🌹    🌹   🌹  
